In [ ]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 66.9 MB/s eta 0:00:00


In [ ]:
import onnx
import numpy as np

from onnx import TensorProto, numpy_helper
from onnx.helper import (
    make_model, make_node, make_graph,
    make_tensor_value_info)
from onnx.checker import check_model

X = make_tensor_value_info('X', TensorProto.FLOAT, [8])
O = make_tensor_value_info('O', TensorProto.FLOAT, [8])

N=[]

value = np.array([[.92, 0, 0, 0, 0, 0, 0, 0],[.51, 0, 0, 0, 0, 0, 0, 0],[.1, 0, 0, 0, 0, 0, 0, 0] ,[.3, 0, 0, 0, 0, 0, 0, 0] ,[.2, 0, 0, 0, 0, 0, 0, 0],[.6, 0, 0, 0, 0, 0, 0, 0] ,[.1, 0, 0, 0, 0, 0, 0, 0] ,[.5, 0, 0, 0, 0, 0, 0, 0]],  dtype=np.float32)
Lf = numpy_helper.from_array(value, name='Lf')

value = np.array([np.random.uniform(0.0, 1.0, 8),np.random.uniform(0.0, 1.0, 8), np.random.uniform(0.0, 1.0, 8), np.random.uniform(0.0, 1.0, 8), np.random.uniform(0.0, 1.0, 8), np.random.uniform(0.0, 1.0, 8), np.random.uniform(0.0, 1.0, 8), np.random.uniform(0.0, 1.0, 8)])
, dtype=np.float32)
L = numpy_helper.from_array(value, name='L')
node = make_node('Gemm', ['X', 'L'], ['XL'])
N.append(node)
node = make_node(
    "Relu",
    inputs=["XL"],
    outputs=["Y1"],
)
N.append(node)

node = make_node('Gemm', ['Y1','Lf'], ['O'])
N.append(node)

graph = make_graph(N, 'lr', [X], [O], [L, Lf])
onnx_model = make_model(graph)
check_model(onnx_model)

print(onnx_model)


ir_version: 9
graph {
  node {
    input: "X"
    input: "L"
    output: "XL"
    op_type: "Gemm"
  }
  node {
    input: "XL"
    output: "Y1"
    op_type: "Relu"
  }
  node {
    input: "Y1"
    input: "Lf"
    output: "O"
    op_type: "Gemm"
  }
  name: "lr"
  initializer {
    dims: 8
    dims: 8
    data_type: 11
    name: "L"
    raw_data: "\306;\r\"\017\221\350?D\360bK!\236\333?\334\234, b\025\302?\000\026c\215\232^\334?,b\250\033m\250\331?\271\3563\242]\343\354?\330d\007\323\354P\317?\251#\326-\233\025\342?\"\014\n0\2076\356?8\255\342\010xF\263??\312\023\t\364\230\351?\371\207:\263T7\352?\240\320\343\007\265l\273?B\033\020B\026\030\324?\337vh\315\225\315\356?\222\307\356$T\334\343?M(\351f%\356\341?\300\325\243\372\"L\213?\237z\232\005]@\356?\200\367(\376\324\216\262?[\261\033zL\374\342?\300\202\3627*\306\207?\304\"h\004VF\321?p\370\267\371h\353\321?\204\2731\313\211,\356?\032\365\024\216\266\350\324?\0307,\177Y\311\302?\n\311|]\'{\352?\000\003\213\237E(\270?\000\014\035h\304\02

In [ ]:
with open("nn81.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())